In [1]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
import torch
import torch.optim as optim
import pandas as pd
import config 

from iTransformer import iTransformer
from utils import data_handling, training_functions, helpers
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm


# Define constants
FOUR_WEEKS = -24*7*4
EPOCH = 15
WINDOW_SIZE= 96
PRED_LENGTH = (96)

# define output file
metrics_output_path = config.CONFIG_OUTPUT_PATH["itransformer"] / "itransformer_results_baseline.csv"
target_dataset = ["ELD", "Bavaria", "GP2"]

# Load previous results if present
try:
    results_df = pd.read_csv(metrics_output_path, index_col=[0, 1, 2])
except FileNotFoundError:
    metrics = ["MSE", "MAE"]
    learning_scenarios = ["Zero-Shot", "four_weeks_tl", "full_tl", "full_training", "four_weeks_training"]
    index = pd.MultiIndex.from_product([target_dataset, learning_scenarios, metrics], names=["Target", "Learning_scenario", "Metric"])
    results_df = pd.DataFrame(columns=["iTransformer"], index=index)

# Helper functions
def update_metrics(target_dataset, model_name, learning_scenario, mae, mse):
    results_df.loc[(target_dataset, learning_scenario, "MAE"), model_name] = mae
    results_df.loc[(target_dataset, learning_scenario, "MSE"), model_name] = mse


print("Import succesfull")

Import succesfull


# Do baseline predicitons on all 3 target test splits using iTransformer

This notebook trains the iTransformer model on all 3 datasets for the four_week and the full_training setups.
The best model on validation MSE is saved for transfer learning.

In [2]:
# use electricity dataset
data_dict = data_handling.load_electricity()

electricity = {}
electricity["dataloader_train"], electricity["dataloader_validation"], electricity["dataloader_test"] = data_handling.convert_data(data_dict, WINDOW_SIZE, PRED_LENGTH)
data_dict["train"].shape

# create a smaller subset of the train dataset
electricity["4_weeks_train"] = data_dict["train"][FOUR_WEEKS:,:]
electricity["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(electricity["4_weeks_train"] , WINDOW_SIZE, PRED_LENGTH)
electricity["4_weeks_train"] = data_handling.DataLoader(electricity["4_weeks_train"] , batch_size=32, shuffle=True)

Feature batch shape: torch.Size([32, 96, 348])


In [3]:
# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
genome_project = {}
genome_project["dataloader_train"], genome_project["dataloader_validation"], genome_project["dataloader_test"] = data_handling.convert_data(gp_dict, WINDOW_SIZE, PRED_LENGTH)

# create a smaller subset of the train dataset
genome_project["4_weeks_train"] = gp_dict["train"][FOUR_WEEKS:,:]
genome_project["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(genome_project["4_weeks_train"] , WINDOW_SIZE, PRED_LENGTH)
genome_project["4_weeks_train"] = data_handling.DataLoader(genome_project["4_weeks_train"] , batch_size=32, shuffle=True)


Feature batch shape: torch.Size([32, 96, 1454])


In [4]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
bavaria = {}
bavaria["dataloader_train"], bavaria["dataloader_validation"], bavaria["dataloader_test"] = data_handling.convert_data(data_dict, WINDOW_SIZE, PRED_LENGTH)

# create a smaller subset of the train dataset
bavaria["4_weeks_train"] = data_dict["train"][FOUR_WEEKS:,:]
bavaria["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(bavaria["4_weeks_train"] , WINDOW_SIZE, PRED_LENGTH)
bavaria["4_weeks_train"] = data_handling.DataLoader(bavaria["4_weeks_train"] , batch_size=32, shuffle=True)

Feature batch shape: torch.Size([32, 96, 59])


In [5]:
# run experiment for each dataset and save model and evaluation metrics
dataset_dict = {
                "ELD": electricity,
                "Bavaria": bavaria,
                "GP2" : genome_project,                
                }


def train_and_evaluate(dataset_dict, dataset_name, full_dataset= True, epoch=15):
    """
    Instantiate, train, and evaluate the setup specified from the input
    Results are saved as csv 
    Checkpoints are saved and resumed
    """
    if full_dataset == False:
        print("Selecting 4 week dataset")
        training_dataloader = dataset_dict["4_weeks_train"]
    else:
        training_dataloader = dataset_dict["dataloader_train"]
        
    inputs, _ = next(iter(training_dataloader))
    num_variates = inputs.size(2)
    
    # define parameters and create config 
    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.25, 'ff_mult': 4, 'ff_dropout': 0.25, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}


    model_config = {
        'num_variates': num_variates,
        'lookback_len': WINDOW_SIZE,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': PRED_LENGTH,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': True,
        'flash_attn': True
    }

    # select available deviec
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {device}")

    # defining all needed instances
    model = iTransformer(**model_config).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    #writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["itransformer"] / dataset_name)

    # run model training as mentioned in the original paper
    if full_dataset == False:
        checkpoint_path = config.CONFIG_MODEL_LOCATION["itransformer"] / dataset_name / f"{dataset_name}_4_weeks_best_val_loss.pt"
    else:
        checkpoint_path = config.CONFIG_MODEL_LOCATION["itransformer"] / dataset_name / f"{dataset_name}_full_dataset_best_val_loss.pt"


    # load model with best validaiton mse
    try:
        checkpoint = torch.load(checkpoint_path)
        model = iTransformer(**model_config).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        epoch = EPOCH - checkpoint["epoch"]
        if epoch == 0:
            print("Model is already trained for 15 epochs.")
            return None
    except:
        print("Training from scratch.")

    _, best_model = training_functions.train_one_epoch(epoch=epoch, model=model, device=device, \
                                                                   dataloader_train=training_dataloader,    \
                                                dataloader_validation=dataset_dict["dataloader_validation"],\
                                                optimizer=optimizer, checkpoint_path=checkpoint_path,       \
                                                save_model=True, validate=True)


    # predict on test set
    metrics = helpers.full_eval(best_model, dataset_dict["dataloader_test"], device)

    if full_dataset == False:
        update_metrics(dataset_name, "iTransformer", "four_weeks_training", metrics[1], metrics[0])
    else:
        update_metrics(dataset_name, "iTransformer", "full_training", metrics[1], metrics[0])


In [6]:
# Train on small target datasplit
for key, value in dataset_dict.items():
    train_and_evaluate(value, key, full_dataset=False, epoch=EPOCH)

results_df.to_csv(metrics_output_path)

Selecting 4 week dataset
Using device: cuda:1
Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda
Training from scratch.


Epoch: Validating: 100%|██████████| 21/21 [00:06<00:00,  3.45it/s]


Validation metrics: {'mse': tensor(0.4310, device='cuda:1')}
Checkpointing succesfull after epoch 1


Epoch: Validating: 100%|██████████| 21/21 [00:07<00:00,  2.64it/s]


Validation metrics: {'mse': tensor(0.3208, device='cuda:1')}
Checkpointing succesfull after epoch 2


...

Epoch: Validating: 100%|██████████| 40/40 [00:04<00:00,  8.53it/s]


Validation metrics: {'mse': tensor(0.3656, device='cuda:1')}
Checkpointing succesfull after epoch 14


Epoch: Validating: 100%|██████████| 40/40 [00:03<00:00, 11.53it/s]


Validation metrics: {'mse': tensor(0.3687, device='cuda:1')}
Checkpointing succesfull after epoch 15


Epoch: Validating: 100%|██████████| 86/86 [00:23<00:00,  3.60it/s]


In [7]:
# Train on full target train datasplit
for key, value in dataset_dict.items():
    print(key)
    train_and_evaluate(value, key, full_dataset=True, epoch=EPOCH)
results_df.to_csv(metrics_output_path)

ELD
Using device: cuda:1
Training from scratch.


Epoch: Validating: 100%|██████████| 21/21 [00:04<00:00,  5.14it/s]


Validation metrics: {'mse': tensor(0.2502, device='cuda:1')}
Checkpointing succesfull after epoch 1


Epoch: Validating: 100%|██████████| 21/21 [00:02<00:00,  9.81it/s] 


Validation metrics: {'mse': tensor(0.2252, device='cuda:1')}
Checkpointing succesfull after epoch 2


...

Epoch: Validating: 100%|██████████| 40/40 [00:14<00:00,  2.80it/s]


Validation metrics: {'mse': tensor(0.3280, device='cuda:1')}
Checkpointing succesfull after epoch 14


Epoch: Validating: 100%|██████████| 40/40 [00:03<00:00, 10.06it/s]


Validation metrics: {'mse': tensor(0.3350, device='cuda:1')}
Checkpointing succesfull after epoch 15


Epoch: Validating: 100%|██████████| 86/86 [00:16<00:00,  5.35it/s]
